In [2]:
# !pip install torch==2.1.0+cu118 torchvision xformers --extra-index-url https://download.pytorch.org/whl/cu118


In [3]:
# !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple xformers==0.0.22.post7 
# xformers==0.0.22.post7   torchaudio 

In [4]:
# read book1.txt into a list of lines. The book is one of reference of NLPEC test.
# !pip uninstall xformers torch torchvision -y

# Load Language Model

In [5]:
import json
import torch
import pandas as pd
import numpy as np
import faiss
import pickle

# import streamlit as st
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation.utils import GenerationConfig

In [6]:
# model = AutoModelForCausalLM.from_pretrained("./model/Baichuan2-7B-Chat-4bits", device_map="auto", trust_remote_code=True)
# model.generation_config = GenerationConfig.from_pretrained(
#        "./model/Baichuan2-7B-Chat-4bits"
#     )
# tokenizer = AutoTokenizer.from_pretrained("./model/Baichuan2-7B-Chat-4bits", trust_remote_code=True)
# model = model.eval()



# Prepare the dataset

In [7]:
import json 
questions = []
with open('./data/NLPEC/train.json') as f:
  for line in f:
    # Remove newline character
    line = line.strip()
    
    # Load the JSON data
    data = json.loads(line)
    
    # Now you can access the data
    # print(data['key1']) 
    q = data['questionText']
    o = data['option']
    a = data['answer']
    a_alpha = ""
    if a[0] == '1':
      a_alpha = "A"
    if a[0] == '2':
      a_alpha = "B"
    if a[0] == '3':
      a_alpha = "C"
    if a[0] == '4':
      a_alpha = "D"
    if a[0] == '5':
      a_alpha = "E"
    
    questions.append((q,o,a_alpha))

def accuracy(result):
    return len([i for i in result if i[2].strip() == i[3].strip()]) / len(result)

def instruction_following(result):
    return len([i for i in result if i[3].strip() in ['A','B','C','D','E']]) / len(result)

In [8]:
from tqdm import tqdm
import random

def get_context(query):
    tv = to_embeddings(t2v_model, [query])
    dis, ind = index.search(tv, 10)
    qa_id_list = list(ind[0])
    context = kb_dict[qa_id_list[0]]['q']
    # print(context)
    context_embeding = to_embeddings(t2v_model, [context])
    # return calc_similarity_scores(t2v_model, [query], [context])
    if calc_similarity_scores(t2v_model, [query], [context]) > 0.983:
        return context
    else:
        return ""


# def query_rag_model(query):
#     context = get_context(query)
#     inputs = tokenizer(context, return_tensors='pt')
#     inputs = inputs.to('cuda:0')
#     pred = model.generate(**inputs, max_new_tokens=1280, repetition_penalty=1.1, temperature=0.001)
#     response = tokenizer.decode(pred.cpu()[0], skip_special_tokens=True)
#     # print(response)

    return response[len(context):]

def query_model(query, temperature=0.001):
    # context = get_context(query)
    inputs = tokenizer(query, return_tensors='pt')
    inputs = inputs.to('cuda:0')
    pred = model.generate(**inputs, max_new_tokens=1280, repetition_penalty=1.1, temperature=temperature)
    response = tokenizer.decode(pred.cpu()[0], skip_special_tokens=True)
    # print(response)

    return response[len(query):]



def answer_questions(question_set, prompt, temperature):
    result = []
    # subset = random.sample(questions, 10) 

    for q, o, a in tqdm(question_set):
    # for q, o, a in questions:
        # question = prompt + q.strip() + ' || 选项：'
        # question = prompt_before + q.strip() + ' || 选项：'
        pro = prompt.replace('__question__', q.strip())
        opt_str = ""
        idx = ['A', 'B', 'C', 'D', 'E']
        i = 0
        for opt in o:
            opt_str = opt_str + idx[i] + '. ' + opt.strip() + ' '
            i = i +1
        pro = pro.replace('__options__', opt_str)
        question = pro
        # print(question)
        sa = query_model(question,temperature)
        # print(sa)
        result.append((q, o, a, sa))
        # print(result)
    
    return accuracy(result), instruction_following(result),result


In [9]:
len(questions)

18703

# book 教材资料

In [10]:
import re

# Open the file and read the contents
with open('book1.txt', 'r', encoding='utf-8') as f:
    contents = f.readlines()

# Define the patterns to match
patterns = ['^\([一二三四五六七八九十]+\)', '^[0-9]+\.', '^\([0-9]+\)','^[一二三四五六七八九十]+']

# Loop through the contents and find the start line number of the patterns
start_line_numbers = [0]
for i, line in enumerate(contents):
    for pattern in patterns:
        if re.match(pattern, line):
            start_line_numbers.append(i)
            break

# Print the start line numbers
# print(start_line_numbers)
processing_blocks = start_line_numbers

vectors_content = []

for idx in range(len(processing_blocks)):
    if idx > 0:
        ln_start = idx - 1
    else:
        ln_start = 0
    if idx > len(processing_blocks) - 2:
        ln_end =  len(processing_blocks) - 1
    else:
        ln_end = idx + 1
    # print(idx, ln_start, ln_end)
    
    context_content = ''.join(contents[processing_blocks[ln_start]:processing_blocks[ln_end]]).replace('\n', '').replace(' ', '')
    # print(context_content, len(context_content))
    vectors_content.append(context_content)


In [11]:
from text2vec import SentenceModel
from text2vec import CosentModel, SentenceBertModel, BertMatchModel
from text2vec import cos_sim, compute_spearmanr, EncoderType, load_text_matching_test_data
t2v_model = SentenceModel('./model/t2v')

2023-11-12 15:49:03.320 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cuda


In [12]:
def dump_obj(obj_name, obj):
    with open('./data/tmp/' + obj_name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def restore_obj(obj_name):
    with open('./data/tmp/' + obj_name + '.pkl', 'rb') as f:
        obj = pickle.load(f)
        return obj
def to_embeddings(model, text):
    sentence_embeddings = model.encode(text)
    return sentence_embeddings
def calc_similarity_scores(model, sents1, sents2):
    # t1 = time.time()
    e1 = model.encode(sents1)
    e2 = model.encode(sents2)
    # spend_time = time.time() - t1
    s = cos_sim(e1, e2)
    sims = []
    for i in range(len(sents1)):
        sims.append(s[i][i])
    sims = np.array(sims)
    return sims

In [13]:
kb_dict = {}
for index in range(len(vectors_content)):
    kb_dict[index] = {
        'q': vectors_content[index].strip()
        }

In [40]:
dump_obj('kb_dict',kb_dict)

In [14]:


data = to_embeddings(t2v_model,list(vectors_content))
data = data.astype(np.float32)

dim = 384
# Create index using index factory
index = faiss.index_factory(dim, "IVF128,PQ16") 

# Training is required for some indexes like IVF
index.train(data) 

# Add vectors to index
index.add(data)
faiss.write_index(index, "index_file.index") #将index保存为index_file.index文件
index = faiss.read_index("index_file.index") #读入index_file.index文件

WARNING clustering 3988 points to 128 centroids: please provide at least 4992 training points
WARNING clustering 3988 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3988 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3988 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3988 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3988 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3988 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3988 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3988 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3988 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 3988 points to 256 centroids: please prov

挑选出有足够的知识库支持的问题，方便做知识增强验证(总共能跳出104个)

In [15]:
counter = 0
subset = []
contextset = []
for i in range(len(questions)):
    text = questions[i][0] + ''.join(questions[i][1])
    context = get_context(text)
    # print(text[0])
    if len(context)>0:
        counter = counter + 1
        subset.append(questions[i])
        contextset.append(context)

In [16]:
subset

[('适用于缓解肌肉、软组织和关节轻中度疼痛的非处方药是',
  ['谷维素', '布洛芬', '对乙酰氨基酚', '氢溴酸山莨菪碱', '双氯芬酸钠二乙胺乳胶剂'],
  'E'),
 ('患者男，70岁，患高血压病10年，2周来反复出现活动后胸痛。检查：血压170／100mmHg，心率90次／分，律齐，血钾偏高。心电图正常，血管彩超提示双侧颈动脉斑块和双侧肾动脉狭窄。最适宜的联合用药方案是',
  ['血管紧张素转换酶抑制剂+β受体阻断剂+噻嗪类利尿剂',
   '钙通道阻滞剂+β受体阻断剂+噻嗪类利尿剂',
   '血管紧张素Ⅱ受体阻断剂+钙通道阻滞剂+β受体阻断剂',
   'a受体阻断剂+钙通道阻滞剂+醛固酮受体阻断剂',
   '血管紧张素Ⅱ受体阻断剂+中枢a受体阻断剂+噻嗪类利尿剂'],
  'B'),
 ('良性前列腺增生症的晚期病人尿频严重、形成慢性尿潴留，可用特拉唑嗪治疗，下列属于其不良反应的是',
  ['性欲减退', '体位性低血压', '胃灼热、恶心', '射精障碍', '轻微腹胀'],
  'B'),
 ('引起乳剂破裂的是',
  ['ζ电位.降低', '油水两相密度差不同造成', '光、热、空气及微生物等的作用', '乳化剂类型改变', '乳化剂失去作用'],
  'E'),
 ('患者男，70岁，患高血压病10年，2周来反复出现活动后胸痛。检查：血压170/100mmHg，心率90次/分，律齐，血钾偏高。心电图正常，血管彩超提示双侧颈动脉斑块和双侧肾动脉狭窄。最适宜的联合用药方案是',
  ['血管紧张素转换酶抑制剂＋β受体阻断剂＋噻嗪类利尿剂',
   '钙通道阻滞剂＋β受体阻断剂＋噻嗪类利尿剂',
   '血管紧张素Ⅱ受体阻断剂＋钙通道阻滞剂＋β受体阻断剂',
   'α受体阻断剂＋钙通道阻滞剂＋醛固酮受体阻断剂',
   '血管紧张素Ⅱ受体阻断剂＋中枢α受体阻断剂＋噻嗪类利尿剂'],
  'B'),
 ('多发生于春夏季节，可反复发作，双眼奇痒，睑结膜有粗大的乳头，角膜缘胶样增生的是',
  ['急性卡他性结膜炎', '流行性结膜炎', '流行性出血性结膜炎', '过敏性结膜炎', '春季卡他性结膜炎'],
  'E'),
 ('危重支气管哮喘疾病的临床表现为',
  ['步行或上楼时气短，可有焦虑，呼吸频率轻度增加，闻及散在

# Hyper Parameters: Temprature

In [23]:
# prompt = """
# 你是一名医学生，正在参加一项重要考试。你需要逐步阅读并思考每个选项后，首先排除明显错误的选项，再从剩余选项中选择出唯一正确答案。答案用数字编号表示，不要使用字母，也不要引用选项原文。如果不知道答案，就尽量猜测给出一个从1到5的数字作为答案。下面是两个例子：
# 题目：书写药历是药师进行规范化药学服务的一项工作，下列内容一般不作为药历内容的是 || 选项: 1. 患者基本信息 2. 用药评价 3. 临床诊断要点 4. 用药记录 5. 患者知情同意书 || 答案: 5\n
# 题目：研究TDM的临床意义不包括 || 选项: 1. 监督临床用药 2. 研究药物在体内的代谢变化 3. 研究治疗无效的原因 4. 确定患者是否按医嘱服药 5. 研究合并用药的影响 || 答案: 3\n
# 题目："""
prompt = """你是一名药剂师，正在参加一项重要考试。选择出唯一正确答案。
题目： __question__  || 选项：__options__ || 答案: """


In [25]:
temp_5 = []
for i in range(10):
    metrics = answer_questions(subset, prompt, 0.5)
    temp_5.append(metrics)

temp_9 = []
for i in range(10):
    metrics = answer_questions(subset, prompt, 0.9)
    temp_9.append(metrics)
    
temp_1 = []
for i in range(10):
    metrics = answer_questions(subset, prompt, 0.001)
    temp_1.append(metrics)


100%|██████████| 100/100 [00:17<00:00,  5.64it/s]


In [26]:

(round(np.mean([i[0] for i in temp_1]),3), round(np.var([i[0] for i in temp_1]),5), round(np.mean([i[1] for i in temp_1]),3), round(np.var([i[1] for i in temp_1]),5)), \
(round(np.mean([i[0] for i in temp_5]),3), round(np.var([i[0] for i in temp_5]),5), round(np.mean([i[1] for i in temp_5]),3), round(np.var([i[1] for i in temp_5]),5)), \
(round(np.mean([i[0] for i in temp_9]),3), round(np.var([i[0] for i in temp_9]),5), round(np.mean([i[1] for i in temp_9]),3), round(np.var([i[1] for i in temp_9]),5))

((0.22, 0.0, 0.41, 0.0),
 (0.184, 0.00052, 0.343, 0.00086),
 (0.149, 0.00159, 0.287, 0.0017))

## 13B model test

In [21]:
model = AutoModelForCausalLM.from_pretrained("./model/Baichuan2-13B-Chat-4bits", device_map="auto", trust_remote_code=True)
model.generation_config = GenerationConfig.from_pretrained(
       "./model/Baichuan2-13B-Chat-4bits"
    )
tokenizer = AutoTokenizer.from_pretrained("./model/Baichuan2-13B-Chat-4bits", trust_remote_code=True)
model = model.eval()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [105]:
len(temp_1)

0

In [ ]:
temp_5 = []
for i in range(10):
    metrics = answer_questions(subset, prompt, 0.5)
    temp_5.append(metrics)

temp_9 = []
for i in range(10):
    metrics = answer_questions(subset, prompt, 0.9)
    temp_9.append(metrics)
    



In [106]:
temp_1 = []
for i in range(10):
    metrics = answer_questions(subset, prompt, 0.01)
    temp_1.append(metrics)

100%|██████████| 100/100 [00:21<00:00,  4.69it/s]


In [107]:


(round(np.mean([i[0] for i in temp_1]),3), round(np.var([i[0] for i in temp_1]),5), round(np.mean([i[1] for i in temp_1]),3), round(np.var([i[1] for i in temp_1]),5)), \
(round(np.mean([i[0] for i in temp_5]),3), round(np.var([i[0] for i in temp_5]),5), round(np.mean([i[1] for i in temp_5]),3), round(np.var([i[1] for i in temp_5]),5)), \
(round(np.mean([i[0] for i in temp_9]),3), round(np.var([i[0] for i in temp_9]),5), round(np.mean([i[1] for i in temp_9]),3), round(np.var([i[1] for i in temp_9]),5))


((0.184, 2e-05, 0.27, 4e-05),
 (0.144, 0.00046, 0.223, 0.00056),
 (0.104, 0.0008, 0.185, 0.00078))

# Prompt Test
## Prompt 1

In [33]:
prompt = """题目： __question__  || 选项：__options__ || 答案: """

In [34]:
temp_1 = []
for i in range(5):
    metrics = answer_questions(subset, prompt, 0.01)
    temp_1.append(metrics)
(round(np.mean([i[0] for i in temp_1]),3), round(np.var([i[0] for i in temp_1]),5), round(np.mean([i[1] for i in temp_1]),3), round(np.var([i[1] for i in temp_1]),5))


100%|██████████| 104/104 [00:23<00:00,  4.37it/s]


(0.221, 0.0, 0.49, 0.0)

## Prompt 2

In [43]:
prompt = """中国执业药师考试题目，根据药学知识进行回答，选择最佳答案。题目: __question__  || 选项: __options__ || 答案: """

In [44]:
temp_1 = []
for i in range(5):
    metrics = answer_questions(subset, prompt, 0.01)
    temp_1.append(metrics)
(round(np.mean([i[0] for i in temp_1]),3), round(np.var([i[0] for i in temp_1]),5), round(np.mean([i[1] for i in temp_1]),3), round(np.var([i[1] for i in temp_1]),5))


100%|██████████| 104/104 [00:21<00:00,  4.90it/s]


(0.231, 0.0, 0.615, 0.0)

## Prompt 3

In [37]:
prompt = """执业药师考试, 考核医学药学知识，请仔细阅读问题和选项后，选择最佳答案。题目:  __question__  || 选项: __options__ || 答案: """

In [38]:
temp_1 = []
for i in range(5):
    metrics = answer_questions(subset, prompt, 0.01)
    temp_1.append(metrics)
(round(np.mean([i[0] for i in temp_1]),3), round(np.var([i[0] for i in temp_1]),5), round(np.mean([i[1] for i in temp_1]),3), round(np.var([i[1] for i in temp_1]),5))


  0%|          | 0/104 [00:00<?, ?it/s]

100%|██████████| 104/104 [00:26<00:00,  3.95it/s]


(0.298, 4e-05, 0.604, 5e-05)

In [119]:
(round(np.mean([i[0] for i in temp_1]),3), round(np.var([i[0] for i in temp_1]),5), round(np.mean([i[1] for i in temp_1]),3), round(np.var([i[1] for i in temp_1]),5))


(0.0, 0.0, 0.0, 0.0)

## Prompt 4 
Example-based Prompts

In [39]:
# prompt = """你是一名优秀的执业药师，正在参加考试。请逐步阅读并思考每个选项后，选择出唯一正确答案。直接回答一个英文字母。不要其他文字输出，不知道的时候就努力猜测回答，不要回复无法判断。下面是两个例子：
# 题目: 书写药历是药师进行规范化药学服务的一项工作，下列内容一般不作为药历内容的是 || 选项: A. 患者基本信息 B. 用药评价 C. 临床诊断要点 D. 用药记录 E. 患者知情同意书 || 答案: E
# 题目: 研究TDM的临床意义不包括 || 选项: A. 监督临床用药 B. 研究药物在体内的代谢变化 C. 研究治疗无效的原因 D. 确定患者是否按医嘱服药 E. 研究合并用药的影响 || 答案: C
# 题目: __question__  || 选项: __options__ || 答案: """
# prompt = """你是一名医学生，正在参加执业药师考试。 你需要逐步阅读并思考每个选项后, 选择唯一正确答案。直接回答A，B，C，D，E。如果不知道答案，那么猜测选择一个给出答案。
# 题目:  __question__  || 选项: __options__ || 答案: """
prompt = """你是一名优秀的执业药师，正在参加考试, 考核医学药学知识，请仔细阅读问题和选项后，选择最佳答案。直接回答一个英文字母。不要其他文字输出。题目:  __question__  || 选项: __options__ || 答案: """

In [40]:
temp_1 = []
for i in range(5):
    metrics = answer_questions(subset, prompt, 0.01)
    temp_1.append(metrics)
(round(np.mean([i[0] for i in temp_1]),3), round(np.var([i[0] for i in temp_1]),5), round(np.mean([i[1] for i in temp_1]),3), round(np.var([i[1] for i in temp_1]),5))


  0%|          | 0/104 [00:00<?, ?it/s]

100%|██████████| 104/104 [00:11<00:00,  8.77it/s]


(0.342, 2e-05, 0.865, 0.0)

## Prompt 5

In [45]:
prompt = """你是一名执业药师，正在参加考试，考不好你会失业, 考核医学药学知识，请仔细阅读问题和选项后，选择最佳答案。直接回答一个英文字母, 不要其他任何输出。题目:  __question__  || 选项: __options__ || 答案: """

In [46]:
temp_1 = []
for i in range(10):
    metrics = answer_questions(subset, prompt, 0.01)
    temp_1.append(metrics)
(round(np.mean([i[0] for i in temp_1]),3), round(np.var([i[0] for i in temp_1]),5), round(np.mean([i[1] for i in temp_1]),3), round(np.var([i[1] for i in temp_1]),5))


100%|██████████| 104/104 [00:12<00:00,  8.61it/s]


(0.37, 2e-05, 0.865, 0.0)

In [55]:
def get_context(query):
    tv = to_embeddings(t2v_model, [query])
    dis, ind = index.search(tv, 10)
    qa_id_list = list(ind[0])
    context = kb_dict[qa_id_list[0]]['q']
    # print(context)
    context_embeding = to_embeddings(t2v_model, [context])
    # return calc_similarity_scores(t2v_model, [query], [context])
    if calc_similarity_scores(t2v_model, [query], [context]) > 0.983:
        print(query)
        print('---------->')
        print(context)
        return context
    else:
        return ""

In [ ]:
counter = 0
subset = []
contextset = []
for i in range(len(questions)):
    text = questions[i][0] + ''.join(questions[i][1])
    context = get_context(text)
    # print(text[0])
    if len(context)>0:
        counter = counter + 1
        subset.append(questions[i])
        contextset.append(context)


In [65]:
import re

def remove_patterns(text, patterns):
    for pattern in patterns:
        text = re.sub(pattern, '', text)
    return text

patterns = ['\([一二三四五六七八九十]+\)', '[0-9]+\.', '\([0-9]+\)','[一二三四五六七八九十]+']

# Loop through the contents and find the start line number of the patterns
# start_line_numbers = [0]
# for i, line in enumerate(contents):
#     for pattern in patterns:
#         if re.match(pattern, line):
#             start_line_numbers.append(i)
#             break


In [67]:
contextset2 = [remove_patterns(i, patterns) for i in contextset]

## RAG Promopt

In [90]:

def answer_questions_with_context(question_set, context_set, prompt, temperature):
    result = []
    # subset = random.sample(questions, 10) 
    k = 0
    for q, o, a in tqdm(question_set):
        pro = prompt.replace('__context__', context_set[k])
        k = k+1
        pro = pro.replace('__question__', q.strip())
        opt_str = ""
        idx = ['A', 'B', 'C', 'D', 'E']
        i = 0
        for opt in o:
            opt_str = opt_str + idx[i] + '. ' + opt.strip() + ' '
            i = i +1
        pro = pro.replace('__options__', opt_str)
        question = pro
        # print('-------')
        # print(question)
        sa = query_model(question,temperature)
        # print(sa)
        # print('----标准答案---', a)
        result.append((q, o, a, sa))
        # print(result)
    
    return accuracy(result), instruction_following(result),result


prompt = """你是一名优秀的执业药师，正在参加考试, 考核医学药学知识，请仔细阅读参考知识、问题和选项，选择最佳答案。直接回答一个英文字母。不要其他文字输出。 参考知识:  __context__  || 题目:  __question__  || 选项: __options__ || 答案: """

In [91]:
prompt = """你是一名优秀的执业药师，正在参加考试, 考核医学药学知识，请仔细阅读参考知识、问题和选项，选择最佳答案。直接回答一个英文字母。不要其他文字输出。 参考知识:  __context__  || 题目:  __question__  || 选项: __options__ || 答案: """

In [87]:
# prompt = """题目： __question__  || 选项：__options__ || 答案:"""
# prompt = """你是一名优秀的执业药师 请仔细阅读参考知识、问题和选项, 完成单选题,直接给出答案。 参考知识:  __context__  || 题目:  __question__  || 选项: __options__ || 答案: """

In [92]:
temp_1 = []
for i in range(5):
    metrics = answer_questions_with_context(subset, contextset2, prompt, 0.01)
    temp_1.append(metrics)
(round(np.mean([i[0] for i in temp_1]),3), round(np.var([i[0] for i in temp_1]),5), round(np.mean([i[1] for i in temp_1]),3), round(np.var([i[1] for i in temp_1]),5))


100%|██████████| 104/104 [00:22<00:00,  4.70it/s]


(0.429, 2e-05, 0.688, 2e-05)